In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8553040939531623869
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1745276110
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14226764175146375590
 physical_device_desc: "device: 0, name: NVIDIA RTX A2000 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [4]:
import pandas as pd
#import pandas_profiling
import numpy as np
import cupy as cp
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.metrics import mean_squared_error

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import optuna.integration.lightgbm as lgb
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from xgboost.sklearn import XGBClassifier


print("OK\n")

OK



In [5]:
app_train_origin = pd.read_csv('./home-credit-default-risk/exports/app_train.csv')
app_test_origin =  pd.read_csv('./home-credit-default-risk/exports/app_test.csv')
#bureau_origin = pd.read_csv('./home-credit-default-risk/bureau.csv')
#bureau_balance_origin = pd.read_csv('./home-credit-default-risk/bureau_balance.csv')
pos_cash_balance_origin = pd.read_csv('./home-credit-default-risk/POS_CASH_balance.csv')
#credit_card_balance_origin = pd.read_csv('./home-credit-default-risk/credit_card_balance.csv')
previous_application_origin = pd.read_csv('./home-credit-default-risk/previous_application.csv')
installments_payments_origin = pd.read_csv('./home-credit-default-risk/installments_payments.csv')
# custo data
app_train_loan_from_hc_only_cash_origin = pd.read_csv('./home-credit-default-risk/datasets/app_train_loan_from_hc_only_cash.csv')
app_test_loan_from_hc_only_cash_origin = pd.read_csv('./home-credit-default-risk/datasets/app_test_loan_from_hc_only_cash.csv')
app_train_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
app_test_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
app_train_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_annuity_to_max_installment_ratio.csv')
app_test_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_annuity_to_max_installment_ratio.csv')

In [6]:
app_train = app_train_origin.copy()
app_test = app_test_origin.copy()
pos_cash_balance = pos_cash_balance_origin.copy()
previous_application = previous_application_origin.copy()
installments_payments = installments_payments_origin.copy()
app_train_cash = app_train_loan_from_hc_only_cash_origin.copy()
app_test_cash = app_test_loan_from_hc_only_cash_origin.copy()
app_train_1000_mean = app_train_1000_mean_origin.copy()
app_test_1000_mean = app_test_1000_mean_origin.copy()
app_train_max_installment_ratio = app_train_max_installment_ratio_origin.copy()
app_test_max_installment_ratio = app_test_max_installment_ratio_origin.copy()

### applicationの複製

In [7]:
#app_train = app_train[['SK_ID_CURR','TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']].copy()
#app_test = app_test[['SK_ID_CURR','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']].copy()
app_train = app_train[['SK_ID_CURR','TARGET']].copy()
app_test = app_test[['SK_ID_CURR']].copy()
len(app_test)

48744

In [8]:
app_train.head()

,SK_ID_CURR,TARGET
0,100002,1
1,100003,0
2,100004,0
3,100006,0
4,100007,0


### 申し込み月におけるCssh loanの情報をマージ

In [9]:
app_train_cash = app_train_cash.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_cash = app_test_cash.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
len(app_train_cash)

34012

In [10]:
# trainへマージ
app_train = pd.merge(app_train, app_train_cash, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, app_test_cash, on='SK_ID_CURR', how='left')
len(app_train)

307511

### installment_payment_ratio_1000_mean_meanをマージ

In [11]:
app_train_1000_mean = app_train_1000_mean.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_1000_mean = app_test_1000_mean.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_train_1000_mean.head()

,SK_ID_CURR,installment_payment_ratio_1000_mean_mean
0,100003,0.000000
1,100006,0.000000
2,100007,0.000000
3,100008,0.000000
4,100011,-7066.669355


In [12]:
#app_train_1000_mean.drop(['sk_id_prev'], axis=1, inplace=True)
#app_test_1000_mean.drop(['sk_id_prev'], axis=1, inplace=True)
#app_train_1000_mean.head()

In [13]:
# trainへマージ
app_train = pd.merge(app_train, app_train_1000_mean, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, app_test_1000_mean, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [14]:
app_train['installment_payment_ratio_1000_mean_mean']

0                 NaN
1            0.000000
2                 NaN
3            0.000000
4            0.000000
             ...     
307506            NaN
307507            NaN
307508            NaN
307509            NaN
307510    9014.333365
Name: installment_payment_ratio_1000_mean_mean, Length: 307511, dtype: float64

In [15]:
app_train['installment_payment_ratio_1000_mean_mean'].isnull().sum()

193123

In [16]:
#app_train['amt_annuity'].fillna(0, inplace=True)
#app_test['amt_annuity'].fillna(0, inplace=True)

In [17]:
app_train['amt_annuity'].isnull().sum()

273499

### annuity_to_max_installment_ratioをマージ

In [18]:
app_train_max_installment_ratio = app_train_max_installment_ratio.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_max_installment_ratio = app_test_max_installment_ratio.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_max_installment_ratio.head()

,SK_ID_CURR,annuity_to_max_installment_ratio
0,100066,-3217.4444
1,100090,-765.8182
2,100168,-1174.5000
3,100223,-1884.7895
4,100232,-1269.2353


In [19]:
# trainへマージ
app_train = pd.merge(app_train, app_train_max_installment_ratio, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, app_test_max_installment_ratio, on='SK_ID_CURR', how='left')
len(app_train)

307511

### AMT_ANNUITY(ローン支払い額)

In [20]:
max(app_train['amt_annuity'])

nan

In [21]:
min(app_train['amt_annuity'])

nan

In [22]:
app_train[['amt_annuity', 'TARGET']].corr()

,amt_annuity,TARGET
amt_annuity,1.000000,-0.036369
TARGET,-0.036369,1.000000


### AMT_APPLICATION|借り入れ申し込み金額|

In [23]:
#app_train['amt_application'].fillna(0, inplace=True)
#app_test['amt_application'].fillna(0, inplace=True)

In [24]:
app_train[['amt_application', 'TARGET']].corr()

,amt_application,TARGET
amt_application,1.00000,-0.01506
TARGET,-0.01506,1.00000


### AMT_CREDIT(借り入れ承認金額(最終的な貸付金額))

In [25]:
max(app_train['amt_credit'])

nan

In [26]:
min(app_train['amt_credit'])

nan

In [27]:
app_train[['amt_credit', 'TARGET']].corr()

,amt_credit,TARGET
amt_credit,1.000000,-0.008921
TARGET,-0.008921,1.000000


### AMT_DOWN_PAYMENT(頭金) Cash loans の場合は不要?

In [28]:
app_train[['amt_down_payment', 'TARGET']].corr()

,amt_down_payment,TARGET
amt_down_payment,NaN,NaN
TARGET,NaN,1.0


In [29]:
app_train.drop(['amt_down_payment'], axis=1, inplace=True)
app_test.drop(['amt_down_payment'], axis=1, inplace=True)

### AMT_GOODS_PRICE(クライアントが要求した商品価格)

In [30]:
app_train[['amt_goods_price', 'TARGET']].corr()

,amt_goods_price,TARGET
amt_goods_price,1.00000,-0.01506
TARGET,-0.01506,1.00000


### WEEKDAY_APPR_PROCESS_START(申請した曜日)

In [31]:
app_train['weekday_appr_process_start'].replace(['TUESDAY', 'MONDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY'], [1, 2, 3, 4, 5, 6, 7], inplace=True)
app_test['weekday_appr_process_start'].replace(['TUESDAY', 'MONDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY'], [1, 2, 3, 4, 5, 6, 7], inplace=True)

In [32]:
app_train['weekday_appr_process_start'].value_counts()

1.0    5847
2.0    5827
3.0    5694
4.0    5428
5.0    5281
6.0    4033
7.0    1902
Name: weekday_appr_process_start, dtype: int64

In [33]:
app_train[['weekday_appr_process_start', 'TARGET']].corr()

,weekday_appr_process_start,TARGET
weekday_appr_process_start,1.000000,0.004927
TARGET,0.004927,1.000000


In [34]:
app_train.drop(['weekday_appr_process_start'], axis=1, inplace=True)
app_test.drop(['weekday_appr_process_start'], axis=1, inplace=True)

### HOUR_APPR_PROCESS_START(クライアントが応募した時間)

In [35]:
app_train['hour_appr_process_start'].value_counts()

10.0    4744
11.0    4218
12.0    3921
13.0    3333
9.0     3130
14.0    2892
15.0    2577
16.0    1992
8.0     1822
17.0    1404
7.0     1163
18.0     848
6.0      665
5.0      386
19.0     384
4.0      217
3.0      140
20.0      94
2.0       29
21.0      28
22.0      18
1.0        4
0.0        2
23.0       1
Name: hour_appr_process_start, dtype: int64

In [36]:
app_train[['hour_appr_process_start', 'TARGET']].corr()

,hour_appr_process_start,TARGET
hour_appr_process_start,1.000000,-0.018993
TARGET,-0.018993,1.000000


### FLAG_LAST_APPL_PER_CONTRACT|最も最新の過去のローン申請だった場合のフラグ(店員による間違いがあるかもしれんけど)|

In [37]:
app_train['flag_last_appl_per_contract'].value_counts()

Y    34012
Name: flag_last_appl_per_contract, dtype: int64

In [38]:
app_train.drop(['flag_last_appl_per_contract'], axis=1, inplace=True)
app_test.drop(['flag_last_appl_per_contract'], axis=1, inplace=True)

### NFLAG_LAST_APPL_IN_DAY(1日の中での最後の申し込みだった場合のフラグ)

In [39]:
app_train['nflag_last_appl_in_day'].value_counts()

1.0    33970
0.0       42
Name: nflag_last_appl_in_day, dtype: int64

In [40]:
app_train[['nflag_last_appl_in_day', 'TARGET']].corr()

,nflag_last_appl_in_day,TARGET
nflag_last_appl_in_day,1.000000,0.003847
TARGET,0.003847,1.000000


### RATE_DOWN_PAYMENT(前払い率)

In [41]:
app_train['rate_down_payment'].value_counts()

0.0    34012
Name: rate_down_payment, dtype: int64

In [42]:
app_train.drop(['rate_down_payment'], axis=1, inplace=True)
app_test.drop(['rate_down_payment'], axis=1, inplace=True)

### RATE_INTEREST_PRIMARY(金利)

In [43]:
app_train['rate_interest_primary'].value_counts()

0.0    34012
Name: rate_interest_primary, dtype: int64

### RATE_INTEREST_PRIVILEGED(金利)

In [44]:
app_train['rate_interest_privileged'].value_counts()

0.0    34012
Name: rate_interest_privileged, dtype: int64

### NAME_CASH_LOAN_PURPOSE(現金融資の目的)

In [45]:
columns = ['XNA', 'Repairs', 'Other', 'Urgent needs', 'Building a house or an annex', 'Buying a used car', 'Medicine', 'Wedding / gift / holiday', 'Furniture', 'Journey', 'Everyday expenses', 'Education', 'Car repairs', 'Payments on other loans', 'Purchase of electronic equipment', 'Buying a new car', 'Buying a home', 'Gasification / water supply', 'Buying a holiday home / land', 'Business development', 'Buying a garage', 'Hobby', 'Refusal to name the goal', 'Money for a third person']
app_train['name_cash_loan_purpose'].replace(columns, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], inplace=True)
app_test['name_cash_loan_purpose'].replace(columns, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], inplace=True)

In [46]:
app_train['name_cash_loan_purpose'].value_counts()

1.0     31474
2.0       917
3.0       505
4.0       423
5.0       100
6.0        89
7.0        70
8.0        64
9.0        50
10.0       48
11.0       44
12.0       38
13.0       36
14.0       30
15.0       29
16.0       21
17.0       18
18.0       16
19.0       14
20.0       13
21.0        6
22.0        4
23.0        2
24.0        1
Name: name_cash_loan_purpose, dtype: int64

In [47]:
app_train[['name_cash_loan_purpose', 'TARGET']].corr()

,name_cash_loan_purpose,TARGET
name_cash_loan_purpose,1.000000,0.030993
TARGET,0.030993,1.000000


### NAME_CONTRACT_STATUS(契約状況)

In [48]:
app_train['name_contract_status'].value_counts()

Approved    34012
Name: name_contract_status, dtype: int64

In [49]:
app_train.drop(['name_contract_status'], axis=1, inplace=True)
app_test.drop(['name_contract_status'], axis=1, inplace=True)

### DAYS_DECISION(今回申込日との日数差)

In [50]:
app_train['days_decision'].value_counts()

-244.0     124
-364.0     108
-245.0     107
-273.0     105
-183.0     104
          ... 
-2327.0      1
-1287.0      1
-2567.0      1
-1220.0      1
-19.0        1
Name: days_decision, Length: 1367, dtype: int64

In [51]:
app_train[['days_decision', 'TARGET']].corr()

,days_decision,TARGET
days_decision,1.000000,0.005374
TARGET,0.005374,1.000000


In [52]:
# days_decisionをDROP
app_train.drop(['days_decision'], axis=1, inplace=True)
app_test.drop(['days_decision'], axis=1, inplace=True)

### NAME_PAYMENT_TYPE(支払い方法)

In [53]:
columns = ['Cash through the bank', 'XNA', 'Non-cash from your account', 'Cashless from the account of the employer']
app_train['name_payment_type'].replace(columns, [1, 2, 3, 4], inplace=True)
app_test['name_payment_type'].replace(columns, [1, 2, 3, 4], inplace=True)

In [54]:
app_train['name_payment_type'].value_counts()

1.0    30767
2.0     3079
3.0      138
4.0       28
Name: name_payment_type, dtype: int64

In [55]:
app_train[['name_payment_type', 'TARGET']].corr()

,name_payment_type,TARGET
name_payment_type,1.000000,0.015109
TARGET,0.015109,1.000000


### CODE_REJECT_REASON(審査落ち理由)

In [56]:
app_train['code_reject_reason'].value_counts()

XAP    34012
Name: code_reject_reason, dtype: int64

In [57]:
app_train.drop(['code_reject_reason'], axis=1, inplace=True)
app_test.drop(['code_reject_reason'], axis=1, inplace=True)

### NAME_TYPE_SUITE(申請時同行者)

In [58]:
columns = ['Unaccompanied', 'Family', 'Spouse, partner', 'Children', 'Other_B', 'Other_A', 'Group of people']
app_train['name_type_suite'].replace(columns, [1, 2, 3, 4, 5, 6, 7], inplace=True)
app_test['name_type_suite'].replace(columns, [1, 2, 3, 4, 5, 6, 7], inplace=True)

In [59]:
app_train['name_type_suite'].value_counts()

1.0    18101
2.0     3267
3.0      946
4.0      312
5.0      159
6.0       57
7.0       23
Name: name_type_suite, dtype: int64

In [60]:
app_train[['name_type_suite', 'TARGET']].corr()

,name_type_suite,TARGET
name_type_suite,1.00000,0.00975
TARGET,0.00975,1.00000


### NAME_CLIENT_TYPE(新規顧客などの顧客タイプ)

In [61]:
columns = ['Repeater', 'Refreshed', 'New', 'XNA']
app_train['name_client_type'].replace(columns, [1, 2, 3, 4], inplace=True)
app_test['name_client_type'].replace(columns, [1, 2, 3, 4], inplace=True)

In [62]:
app_train['name_client_type'].value_counts()

1.0    29422
2.0     3320
3.0     1243
4.0       27
Name: name_client_type, dtype: int64

In [63]:
app_train[['name_client_type', 'TARGET']].corr()

,name_client_type,TARGET
name_client_type,1.00000,0.02332
TARGET,0.02332,1.00000


### NAME_GOODS_CATEGORY(申請した商品の種類)

In [64]:
app_train['name_goods_category'].value_counts()

XNA    34012
Name: name_goods_category, dtype: int64

In [65]:
app_train.drop(['name_goods_category'], axis=1, inplace=True)
app_test.drop(['name_goods_category'], axis=1, inplace=True)

### name_portfolio

In [66]:
app_train['name_portfolio'].value_counts()

Cash    34012
Name: name_portfolio, dtype: int64

In [67]:
app_train.drop(['name_portfolio'], axis=1, inplace=True)
app_test.drop(['name_portfolio'], axis=1, inplace=True)

### name_product_type

In [68]:
columns = ['x-sell', 'walk-in']
app_train['name_product_type'].replace(columns, [1, 2], inplace=True)
app_test['name_product_type'].replace(columns, [1, 2], inplace=True)

In [69]:
app_train[['name_product_type', 'TARGET']].corr()

,name_product_type,TARGET
name_product_type,1.000000,0.046163
TARGET,0.046163,1.000000


In [70]:
app_train['name_product_type'].value_counts()

1.0    30722
2.0     3290
Name: name_product_type, dtype: int64

### channel_type

In [71]:
columns = ['Credit and cash offices', 'AP+ (Cash loan)', 'Contact center', 'Country-wide', 'Stone', 'Regional / Local', 'Channel of corporate sales', 'Car dealer']
app_train['channel_type'].replace(columns, [1, 2, 3, 4, 5, 6, 7, 8], inplace=True)
app_test['channel_type'].replace(columns, [1, 2, 3, 4, 5, 6, 7, 8], inplace=True)

In [72]:
app_train['channel_type'].value_counts()

1.0    25962
2.0     3081
3.0     2516
4.0     1451
5.0      363
6.0      338
7.0      300
8.0        1
Name: channel_type, dtype: int64

In [73]:
app_train[['channel_type', 'TARGET']].corr()

,channel_type,TARGET
channel_type,1.000000,0.019057
TARGET,0.019057,1.000000


### sellerplace_area

In [74]:
app_train['sellerplace_area'].value_counts()

-1.0        27416
 0.0         1550
 4.0          694
 5.0          453
 3.0          289
            ...  
 288.0          1
 2105.0         1
 134.0          1
 13000.0        1
 3102.0         1
Name: sellerplace_area, Length: 345, dtype: int64

In [75]:
app_train[['sellerplace_area', 'TARGET']].corr()

,sellerplace_area,TARGET
sellerplace_area,1.000000,0.011037
TARGET,0.011037,1.000000


### name_seller_industry

In [76]:
columns = ['XNA', 'Consumer electronics', 'Connectivity', 'Furniture', 'Industry', 'Construction', 'Auto technology', 'MLM partners', 'Clothing', 'Jewelry']
app_train['name_seller_industry'].replace(columns, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], inplace=True)
app_test['name_seller_industry'].replace(columns, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], inplace=True)

In [77]:
app_train['name_seller_industry'].value_counts()

1.0     31877
2.0       997
3.0       955
4.0       112
5.0        23
6.0        22
7.0         8
8.0         8
9.0         8
10.0        2
Name: name_seller_industry, dtype: int64

In [78]:
app_train[['name_seller_industry', 'TARGET']].corr()

,name_seller_industry,TARGET
name_seller_industry,1.000000,0.026302
TARGET,0.026302,1.000000


### cnt_payment

In [79]:
app_train['cnt_payment'].value_counts()

24.0    7211
36.0    7080
12.0    4991
48.0    4300
60.0    3796
18.0    3264
30.0    1593
42.0     866
6.0      615
54.0     281
72.0       6
23.0       3
29.0       1
15.0       1
44.0       1
35.0       1
26.0       1
59.0       1
Name: cnt_payment, dtype: int64

In [80]:
app_train[['cnt_payment', 'TARGET']].corr()

,cnt_payment,TARGET
cnt_payment,1.000000,0.072413
TARGET,0.072413,1.000000


### name_yield_group

In [81]:
columns = ['low_normal', 'middle', 'high', 'low_action']
app_train['name_yield_group'].replace(columns, [1, 2, 3, 4], inplace=True)
app_test['name_yield_group'].replace(columns, [1, 2, 3, 4], inplace=True)

In [82]:
app_train['name_yield_group'].value_counts()

1.0    15907
2.0    11925
3.0     3575
4.0     2605
Name: name_yield_group, dtype: int64

In [83]:
app_train[['name_yield_group', 'TARGET']].corr()

,name_yield_group,TARGET
name_yield_group,1.000000,0.018434
TARGET,0.018434,1.000000


### product_combination

In [84]:
columns = ['Cash X-Sell: low', 'Cash X-Sell: middle', 'Cash X-Sell: high', 'Cash Street: low', 'Cash Street: middle', 'Cash Street: high']
app_train['product_combination'].replace(columns, [1, 2, 3, 4, 5, 6], inplace=True)
app_test['product_combination'].replace(columns, [1, 2, 3, 4, 5, 6], inplace=True)

In [85]:
app_train['product_combination'].value_counts()

1.0    17152
2.0    10786
3.0     2473
4.0     1360
5.0     1139
6.0     1102
Name: product_combination, dtype: int64

In [86]:
app_train[['product_combination', 'TARGET']].corr()

,product_combination,TARGET
product_combination,1.000000,0.071361
TARGET,0.071361,1.000000


### days_first_drawing

In [87]:
app_train['days_first_drawing'].value_counts()

365243.0    34000
Name: days_first_drawing, dtype: int64

In [88]:
app_train.drop(['days_first_drawing'], axis=1, inplace=True)
app_test.drop(['days_first_drawing'], axis=1, inplace=True)

In [89]:
### days_first_due

In [90]:
app_train['days_first_due'].value_counts()

-214.0     130
-153.0     109
-334.0     106
-184.0     106
-243.0     104
          ... 
-1352.0      1
-2236.0      1
-1279.0      1
-2297.0      1
-1277.0      1
Name: days_first_due, Length: 1353, dtype: int64

In [91]:
app_train[['days_first_due', 'TARGET']].corr()

,days_first_due,TARGET
days_first_due,1.000000,0.005313
TARGET,0.005313,1.000000


### days_last_due_1st_version

In [92]:
app_train['days_last_due_1st_version'].value_counts()

-8.0       86
-4.0       79
-12.0      78
-10.0      74
-7.0       73
           ..
-100.0      1
-1572.0     1
 1772.0     1
 1741.0     1
 1781.0     1
Name: days_last_due_1st_version, Length: 1840, dtype: int64

In [93]:
app_train[['days_last_due_1st_version', 'TARGET']].corr()

,days_last_due_1st_version,TARGET
days_last_due_1st_version,1.000000,0.074578
TARGET,0.074578,1.000000


In [94]:
### days_last_due

In [95]:
app_train['days_last_due'].value_counts()

 365243.0    29864
-10.0          152
-12.0          145
-8.0           144
-6.0           136
             ...  
-460.0           1
-2116.0          1
-107.0           1
-67.0            1
-255.0           1
Name: days_last_due, Length: 93, dtype: int64

In [96]:
app_train[['days_last_due', 'TARGET']].corr()

,days_last_due,TARGET
days_last_due,1.000000,0.029502
TARGET,0.029502,1.000000


### days_termination

In [97]:
app_train['days_termination'].value_counts()

 365243.0    30457
-2.0           157
-3.0           149
-5.0           146
-9.0           144
             ...  
-1628.0          1
-1504.0          1
-1632.0          1
-1554.0          1
-1534.0          1
Name: days_termination, Length: 65, dtype: int64

In [98]:
app_train[['days_termination', 'TARGET']].corr()

,days_termination,TARGET
days_termination,1.000000,0.026684
TARGET,0.026684,1.000000


### nflag_insured_on_approval

In [99]:
app_train['nflag_insured_on_approval'].value_counts()

1.0    26672
0.0     7328
Name: nflag_insured_on_approval, dtype: int64

In [100]:
app_train[['nflag_insured_on_approval', 'TARGET']].corr()

,nflag_insured_on_approval,TARGET
nflag_insured_on_approval,1.000000,0.012391
TARGET,0.012391,1.000000


### max_sk_dpd

In [101]:
app_train['max_sk_dpd'].value_counts()

0.0       33514
1.0         110
2.0          76
3.0          53
4.0          30
          ...  
2024.0        1
1956.0        1
2186.0        1
74.0          1
117.0         1
Name: max_sk_dpd, Length: 92, dtype: int64

In [102]:
app_train[['max_sk_dpd', 'TARGET']].corr()

,max_sk_dpd,TARGET
max_sk_dpd,1.000000,0.010112
TARGET,0.010112,1.000000


### installment_payment_ratio_1000_mean_mean

In [103]:
app_train['installment_payment_ratio_1000_mean_mean'].value_counts()

 0.000000        57898
 22500.000000      120
 45000.000000       94
 11250.000000       79
 15000.000000       79
                 ...  
-4592.368636         1
-845.156250          1
 20485.198333        1
 6394.845789         1
 9014.333365         1
Name: installment_payment_ratio_1000_mean_mean, Length: 51777, dtype: int64

In [104]:
app_train['installment_payment_ratio_1000_mean_mean']

0                 NaN
1            0.000000
2                 NaN
3            0.000000
4            0.000000
             ...     
307506            NaN
307507            NaN
307508            NaN
307509            NaN
307510    9014.333365
Name: installment_payment_ratio_1000_mean_mean, Length: 307511, dtype: float64

In [105]:
app_train[['installment_payment_ratio_1000_mean_mean', 'TARGET']].corr()

,installment_payment_ratio_1000_mean_mean,TARGET
installment_payment_ratio_1000_mean_mean,1.000000,-0.030569
TARGET,-0.030569,1.000000


In [106]:
len(app_train)

307511

In [107]:
len(app_test)

48744

### CSVをEXPORT

In [108]:
### train
app_train.to_csv(
    path_or_buf="./home-credit-default-risk/exports/hc_only_cash_train.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [109]:
### test
app_test.to_csv(
    path_or_buf="./home-credit-default-risk/exports/hc_only_cash_test.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [110]:
#num_cols = ['max_credit_day_overdue', 'max_days_credit_enddate', 'min_days_enddate_fact', 'max_amt_credit_max_overdue', 'cnt_credit_prolong_in_6month', 'sum_amt_credit_sum', 'sum_amt_credit_sum_debt', 'sum_amt_credit_sum_limit', 'sum_amt_credit_sum_overdue', 'avg_amt_annuity']

data = pd.concat([app_train, app_test], sort=False)
train = data[:len(app_train)]
test = data[len(app_test):]


y_train = train['TARGET']
X_train = app_train.copy().drop('TARGET', axis=1)
X_test = app_test.copy()

# 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(X_train[num_cols])

# 変換後のデータで各列を置換
#X_train[num_cols] = pt.transform(X_train[num_cols])
#X_test[num_cols] = pt.transform(X_test[num_cols])

X_train.head()

,SK_ID_CURR,amt_annuity,amt_application,amt_credit,amt_goods_price,hour_appr_process_start,nflag_last_appl_in_day,rate_interest_primary,rate_interest_privileged,name_cash_loan_purpose,...,name_yield_group,product_combination,days_first_due,days_last_due_1st_version,days_last_due,days_termination,nflag_insured_on_approval,max_sk_dpd,installment_payment_ratio_1000_mean_mean,annuity_to_max_installment_ratio
0,100002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,100004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,100007,16037.64,247500.0,274288.5,247500.0,11.0,1.0,0.0,0.0,1.0,...,2.0,2.0,-344.0,346.0,365243.0,365243.0,1.0,0.0,0.0,-1561.7857


In [111]:
import xgboost as xgb
from sklearn.metrics import log_loss

y_preds = []
models = []
scores = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = columns

params = {
    'booster': 'gbtree'
    ,'objective': 'binary:logistic'
    ,'silent': 1
    ,'random_state': 17
    ,'gpu_id': 0
    ,'tree_method': 'gpu_hist'
    ,'eval_metric': 'auc'
    ,'eta': 0.1
    #,'max_round': 100
    ,'max_depth': 8
    ,'subsample': 0.8
    ,'colsample_state': 71
    ,'colsample_bytree': 1.0
    ,'colsample_bylevel': 0.3
    ,'subsumple': 0.9
    ,'gamma': 0
    ,'lambda': 1
    ,'alpha': 0
    ,'min_child_weight': 1
}

num_round = 50

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)
    
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round, evals=watchlist)
    
    va_pred = model.predict(dvalid)
    score = log_loss(y_val, va_pred)
    oof_train[valid_index] = va_pred
    
    y_pred = model.predict(dtest) 
    
    
    #oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    #y_pred = model.predict(X_test, num_iteration=model.best_iteration)##

    y_preds.append(y_pred)
    models.append(model)
    scores.append(score)

[22:00:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.53718	eval-auc:0.53245
[1]	train-auc:0.54952	eval-auc:0.53631
[2]	train-auc:0.55073	eval-auc:0.53640
[3]	train-auc:0.55256	eval-auc:0.53667
[4]	train-auc:0.55347	eval-auc:0.53600
[5]	train-auc:0.55584	eval-auc:0.53711
[6]	train-auc:0.55643	eval-auc:0.53891
[7]	train-auc:0.55751	eval-auc:0.53883
[8]	train-auc:0.55781	eval-auc:0.53770
[9]	train-auc:0.55862	eval-auc:0.53884
[10]	train-auc:0.55962	eval-auc:0.53828
[11]	train-auc:0.56016	eval-auc:0.53812
[12]	train-auc:0.56306	eval-auc:0.53840
[13]	train-auc:0.56396	eval-auc:0.53849
[14

[12]	train-auc:0.56121	eval-auc:0.54206
[13]	train-auc:0.56219	eval-auc:0.54221
[14]	train-auc:0.56262	eval-auc:0.54264
[15]	train-auc:0.56299	eval-auc:0.54285
[16]	train-auc:0.56373	eval-auc:0.54346
[17]	train-auc:0.56412	eval-auc:0.54367
[18]	train-auc:0.56530	eval-auc:0.54409
[19]	train-auc:0.56509	eval-auc:0.54353
[20]	train-auc:0.56580	eval-auc:0.54441
[21]	train-auc:0.56718	eval-auc:0.54422
[22]	train-auc:0.56771	eval-auc:0.54362
[23]	train-auc:0.56821	eval-auc:0.54453
[24]	train-auc:0.56849	eval-auc:0.54459
[25]	train-auc:0.56948	eval-auc:0.54518
[26]	train-auc:0.56938	eval-auc:0.54524
[27]	train-auc:0.57030	eval-auc:0.54543
[28]	train-auc:0.57099	eval-auc:0.54483
[29]	train-auc:0.57141	eval-auc:0.54403
[30]	train-auc:0.57209	eval-auc:0.54429
[31]	train-auc:0.57210	eval-auc:0.54442
[32]	train-auc:0.57305	eval-auc:0.54388
[33]	train-auc:0.57316	eval-auc:0.54432
[34]	train-auc:0.57381	eval-auc:0.54421
[35]	train-auc:0.57433	eval-auc:0.54390
[36]	train-auc:0.57464	eval-auc:0.54407


In [112]:
y_preds

[array([0.08232608, 0.08232608, 0.12576523, ..., 0.0812972 , 0.0812972 ,
        0.07595982], dtype=float32),
 array([0.07970504, 0.07970504, 0.11108422, ..., 0.08034716, 0.08034716,
        0.07762544], dtype=float32),
 array([0.08117311, 0.08117311, 0.10060679, ..., 0.0808669 , 0.0808669 ,
        0.07965405], dtype=float32),
 array([0.07976759, 0.07976759, 0.0953372 , ..., 0.07851186, 0.07851186,
        0.07651309], dtype=float32),
 array([0.07668689, 0.07668689, 0.10504444, ..., 0.08011776, 0.08011776,
        0.07543893], dtype=float32)]

In [113]:
scores

[0.27462937696194156,
 0.2797467189103116,
 0.2783913456568094,
 0.2793328658330591,
 0.2807752503279045]

In [114]:
# CSV-00
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[0]
submit_all.isnull().sum()

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_00.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [115]:
# CSV-01
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[1]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_01.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [116]:
# CSV-02
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[2]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_02.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [117]:
# CSV-03
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[3]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_03.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [118]:
# CSV-04
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[4]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_04.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [119]:
submit_all_00 = pd.read_csv('./home-credit-default-risk/submit/submit_all_00.csv')
submit_all_01 = pd.read_csv('./home-credit-default-risk/submit/submit_all_01.csv')
submit_all_02 = pd.read_csv('./home-credit-default-risk/submit/submit_all_02.csv')
submit_all_03 = pd.read_csv('./home-credit-default-risk/submit/submit_all_03.csv')
submit_all_04 = pd.read_csv('./home-credit-default-risk/submit/submit_all_04.csv')

In [120]:
submit_all = pd.DataFrame({
    'submit_all_00': submit_all_00['TARGET'].values
    ,'submit_all_01': submit_all_01['TARGET'].values
    ,'submit_all_02': submit_all_02['TARGET'].values
    ,'submit_all_03': submit_all_03['TARGET'].values
    ,'submit_all_04': submit_all_04['TARGET'].values
    })
submit_all.head()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
0,0.082326,0.079705,0.081173,0.079768,0.076687
1,0.082326,0.079705,0.081173,0.079768,0.076687
2,0.125765,0.111084,0.100607,0.095337,0.105044
3,0.082326,0.079705,0.081173,0.079768,0.076687
4,0.082326,0.079705,0.081173,0.079768,0.076687


In [121]:
submit_all.corr()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
submit_all_00,1.000000,0.908244,0.915162,0.854503,0.873539
submit_all_01,0.908244,1.000000,0.945849,0.930975,0.902809
submit_all_02,0.915162,0.945849,1.000000,0.899188,0.908164
submit_all_03,0.854503,0.930975,0.899188,1.000000,0.815570
submit_all_04,0.873539,0.902809,0.908164,0.815570,1.000000


In [122]:
submit_all = app_test[["SK_ID_CURR"]].copy()
submit_all['TARGET'] = (submit_all_00['TARGET'] + submit_all_01['TARGET'] + submit_all_02['TARGET'] + submit_all_03['TARGET'] + submit_all_04['TARGET'])/5
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.079932
1,100005,0.079932
2,100013,0.107568
3,100028,0.079932
4,100038,0.079932


In [123]:
submit_all['TARGET'] = (submit_all['TARGET'] / 5)
submit_all.to_csv('./home-credit-default-risk/submit/submit_hc_only_cash_test.csv', index=False)
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.015986
1,100005,0.015986
2,100013,0.021514
3,100028,0.015986
4,100038,0.015986


In [124]:
submit_all.count()

SK_ID_CURR    48744
TARGET        48744
dtype: int64